In [1]:
# PACKAGES
import requests
from bs4 import BeautifulSoup as bs
import fnmatch
from datetime import datetime
import re
import pandas as pd

In [2]:
response = requests.get("https://www.rsssf.org/tables/2022f.html")
response

<Response [200]>

In [8]:
# PARSING CONTENT TO THE BS FORMAT AND CONVERTING TO STRING
results_bs = bs(response.content, 'html.parser')
results_raw = results_bs.get_text()
# results_raw

In [23]:
# CREATING INITIAL LIST
results_raw_list = results_raw.split("\n")
# results_raw_list

In [12]:
# CREATING A NEW LIST THAT WILL CONTAIN INFORMATION ABOUT MATCH
# AND RELATED PENALTIES IN ONE LINE (IF PENALTIES OCCURED)
results_pen_list = []
for i in range(0,len(results_raw_list)-1):
    if fnmatch.fnmatch(results_raw_list[i], "*aet]*"):
        results_pen_list.append(results_raw_list[i]+results_raw_list[i+1])
    else:
        results_pen_list.append(results_raw_list[i])
# results_pen_list

In [13]:
# PREPARING A SEPARATE AND REFORMATTED POSITION FOR THE WORLD CUP FINAL 
# AS IT HAS BEEN PROVIDED IN A DIFFERENT FORMAT THAN THE REST OF THE MATCHES

# retrieving venue with date and results of the final as they were put
# in separate lines
venue_wrongdate = fnmatch.filter(results_pen_list, "Lusail, December 18, 2022")
final_nodate_novenue = fnmatch.filter(results_pen_list, "Argentina         3-3 France*")
# retrieving venue and date
venue = venue_wrongdate[0].partition(", ")[0]
wrongdate = venue_wrongdate[0].partition(", ")[2]
# converting date to the format used for the remainder of matches
date_elems = datetime.strptime(wrongdate, "%B %d, %Y")
correctdate = datetime.strftime(date_elems, "%d-%m-%y")
# preparing updated line
correctdate_venue = correctdate + ' ' + venue
# two spaces added between venue/date and the remainder of row to support
# pattern recognition later in the code
final = correctdate_venue + '  ' + final_nodate_novenue[0]
final

'18-12-22 Lusail  Argentina         3-3 France           [aet]Argentina won 4-2 on penalties'

In [19]:
# FILTERING OUT IRRELEVANT RECORDS AND CREATING NEW LIST COMPOSED OF
# WORLD CUP MATCHES ALONG WITH INFORMATION ABOUT PENALTIES

match_results = fnmatch.filter(results_pen_list, "??-??-??*")
match_results.append(final)
# match_results

In [25]:
# UPDATING LIST SO THAT:
# 1) COUNTRY NAMES ARE PROTECTED LATER ON WHEN ELEMENTS OF THE LIST 
# WILL BE SPLITTED INTO A FEW SEPARATE LISTS
# Sign '&' will be added for countries which names are composed of
# multiple words e.g. Saudi Arabia -> Saudi&Arabia
# 2) REDUNDANT SPACE (IF EXISTS) AT THE BEGINNING OF DATE IS DELETED
# 3)'USA' converted to 'United&States' to be consistent with the 
# convention used in the project

# pattern1 to identify spaces within country names
pattern1 = '(?<=[a-z]) (?=[A-Z])'
# pattern2 to identify dates with days expressed as a single digit
pattern2 = ' (?=[0-9]-[0-9][0-9]-[0-9][0-9])'
# pattern3 to identify single and multiple spaces to convert them to 
# one sign so that we can split important information
pattern3 = ' {1,}'
match_results_upd = []
for result in match_results:
    if re.search(pattern1, result):
        result = re.sub(pattern1, "&", result)
    if fnmatch.fnmatch(result, '*USA*'):
        result = result.replace('USA', 'United&States')
    if re.search(pattern2, result):
        result = re.sub(pattern2, "", result)
    result = re.sub(pattern3, "#", result)
    match_results_upd.append(result)
# match_results_upd

In [31]:
# RETRIEVAL OF INFORMATION THAT WILL BE USED IN THE DASHBOARD IN THE FORM
# OF LISTS

# Date - format will be changed to YYYY-MM-DD to be consistent with
# the rest of the project
Date = []
# Stage - similarly to the date we will use the naming convention for
# stages that is consistent with the rest of the project
Stage = []
# Home team and away team - we are going to revert changes from the
# previous step and replace "&" with space
Home_Team = []
Away_Team = []
# Goals - data will be split by "-" sign
Home_team_goals = []
Away_team_goals = []
# Match results - if game ended aet (after extra time) then we will use
# the result of penalties (name of the winner is populated right after
# "[aet]" espression). If not, we will look up the goals and determine 
# the winner that way
Match_results = []

for i in range(0, len(match_results_upd)):
    result_upd = match_results_upd[i].split('#')
    date_conver = result_upd[0]
    date_elems = datetime.strptime(date_conver, '%d-%m-%y')
    Date.append(datetime.strftime(date_elems, '%Y-%m-%d'))
    if Date[i] >= '2022-11-20' and Date[i] <= '2022-12-02':
        Stage.append('Group stage')
    elif Date[i] >= '2022-12-03' and Date[i] <= '2022-12-06':
        Stage.append('Round of 16')
    elif Date[i] in ['2022-12-09','2022-12-10']:
        Stage.append('Quarter-finals')
    elif Date[i] in ['2022-12-13','2022-12-14']:
        Stage.append('Semi-finals')
    elif Date[i] == '2022-12-17':
        Stage.append('Third place')
    elif Date[i] == '2022-12-18':
        Stage.append('Final')
    else:
        print("Error related to stages")
    Home_Team.append(result_upd[2].replace("&"," "))
    Away_Team.append(result_upd[4].replace("&"," "))
    Home_team_goals.append(result_upd[3].split("-")[0])
    Away_team_goals.append(result_upd[3].split("-")[1])
    if fnmatch.fnmatch(match_results_upd[i], "*aet]*"):
        penalty_info = match_results_upd[i].split("aet]")[1]
        Match_results.append(penalty_info.split("#")[0])
    else:
        if Home_team_goals[i] > Away_team_goals[i]:
            Match_results.append(Home_Team[i])
        elif Home_team_goals[i] < Away_team_goals[i]:
            Match_results.append(Away_Team[i])
        elif Home_team_goals[i] == Away_team_goals[i]:
            Match_results.append("Draw")
        else:
            print("Error related to match results")

In [32]:
# CREATING DICTIONARY FROM DATA RELEVANT TO THE DASHBOARD 
# AND THEN DATA FRAME BASED ON IT
# Then additional position including information about World Cup 
# Qualification is added for dashboard purposes

wc_results_dict = {
    "Date" : Date,
    "Stage" : Stage,
    "Home Team" : Home_Team,
    "Away Team" : Away_Team,
    "Match_results" : Match_results,
}

wc_results = pd.DataFrame(wc_results_dict)
wc_results.loc[len(wc_results)] = ['2022-06-14','Qualification','','','']
wc_results = wc_results.sort_values("Date", ascending = True)
wc_results

,Date,Stage,Home Team,Away Team,Match_results
64,2022-06-14,Qualification,,,
0,2022-11-20,Group stage,Qatar,Ecuador,Ecuador
6,2022-11-21,Group stage,England,Iran,England
7,2022-11-21,Group stage,United States,Wales,Draw
1,2022-11-21,Group stage,Senegal,Netherlands,Netherlands
...,...,...,...,...,...
59,2022-12-10,Quarter-finals,England,France,France
60,2022-12-13,Semi-finals,Argentina,Croatia,Argentina
61,2022-12-14,Semi-finals,France,Morocco,France
62,2022-12-17,Third place,Croatia,Morocco,Croatia


In [33]:
wc_results.to_csv(r'C:\users\majch\desktop\world_cup_2022_results.csv', index = False)